In [ ]:
# Import necessary libraries
from random import gauss
import matplotlib.pyplot as plt
import numpy as np
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# GARCH(2,2) Model Simulation
This notebook simulates data for a GARCH(2,2) model, visualizes it, and fits the model.

### Mathematical Representation:
$$
a_t = \varepsilon_t \sqrt{\omega + \alpha_1 a_{t-1}^2 + \alpha_2 a_{t-2}^2 + \beta_1 \sigma_{t-1}^2 + \beta_2 \sigma_{t-2}^2}
$$

where:
- $a_0, a_1 \sim \mathcal{N}(0,1)$
- $\sigma_0 =1$, $\sigma_1 = 1$
- $\varepsilon_t \sim \mathcal{N}(0,1)$

In [ ]:
# Parameters for the GARCH(2,2) process
n = 1000  # number of data points
omega = 0.5
alpha_1, alpha_2 = 0.1, 0.2
beta_1, beta_2 = 0.3, 0.4
test_size = int(n * 0.1)  # test set size (10%)

# Generate time series data using GARCH(2,2) model
series = [gauss(0, 1), gauss(0, 1)]  # initial values
vols = [1, 1]  # initial volatilities

for _ in range(n):
    new_vol = np.sqrt(omega + alpha_1 * series[-1]**2 + alpha_2 * series[-2]**2 + beta_1 * vols[-1]**2 + beta_2 * vols[-2]**2)
    new_val = gauss(0, 1) * new_vol
    vols.append(new_vol)
    series.append(new_val)

In [ ]:
# Plotting the generated GARCH(2,2) data and volatility
plt.figure(figsize=(10, 4))
plt.plot(series, label='Simulated Data')
plt.title('Simulated GARCH(2,2) Data', fontsize=20)
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(vols, color='orange', label='Volatility')
plt.title('Data Volatility', fontsize=20)
plt.xlabel('Time')
plt.ylabel('Volatility')
plt.legend()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(series, label='Data')
plt.plot(vols, color='red', label='Volatility')
plt.title('Data and Volatility', fontsize=20)
plt.xlabel('Time')
plt.ylabel('Values/Volatility')
plt.legend()
plt.show()

## PACF plot for squared series values

In [ ]:
plot_pacf(np.array(series)**2)
plt.title('PACF Plot of Squared Series')
plt.show()

## Fit the GARCH Model

In [ ]:
# Split the data into training and test sets
train, test = series[:-test_size], series[-test_size:]

# Fit the GARCH(2,2) model on the training set
model = arch_model(train, p=2, q=2)
model_fit = model.fit()
model_fit.summary()

## Prediction on test data

In [ ]:
predictions = model_fit.forecast(horizon=test_size)

# Plot true vs predicted volatility
plt.figure(figsize=(10, 4))
plt.plot(vols[-test_size:], label='True Volatility')
plt.plot(np.sqrt(predictions.variance.values[-1, :]), label='Predicted Volatility', color='green')
plt.title('Volatility Prediction', fontsize=20)
plt.xlabel('Time')
plt.ylabel('Volatility')
plt.legend()
plt.show()

## Long-term forecast for volatility

In [ ]:
predictions_long_term = model_fit.forecast(horizon=1000)
plt.figure(figsize=(10, 4))
plt.plot(vols[-test_size:], label='True Volatility')
plt.plot(np.sqrt(predictions_long_term.variance.values[-1, :]), label='Predicted Long-Term Volatility', color='purple')
plt.title('Long-Term Volatility Prediction', fontsize=20)
plt.xlabel('Time')
plt.ylabel('Volatility')
plt.legend()
plt.show()

## Rolling forecast to continuously update model on new data

In [ ]:
rolling_predictions = []
for i in range(test_size):
    train_set = series[:-(test_size - i)]
    rolling_model = arch_model(train_set, p=2, q=2)
    rolling_model_fit = rolling_model.fit(disp='off')
    pred = rolling_model_fit.forecast(horizon=1)
    rolling_predictions.append(np.sqrt(pred.variance.values[-1, :][0]))

# Plot rolling forecast results
plt.figure(figsize=(10, 4))
plt.plot(vols[-test_size:], label='True Volatility')
plt.plot(rolling_predictions, label='Rolling Forecast Volatility', color='cyan')
plt.title('Volatility Prediction - Rolling Forecast', fontsize=20)
plt.xlabel('Time')
plt.ylabel('Volatility')
plt.legend()
plt.show()